In [1]:
# Install required libraries
!pip install yfinance timesfm

  Using cached nvidia_nvjitlink_cu12-12.6.20-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.

In [4]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=8cb7c6d484bd5b91d79ed9c494dbaee75b881fb1999d6d674fbd0522150936b8
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [1]:
import os
import time
import timesfm
from collections import defaultdict
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [5]:
# Load TimesFM model
timesfm_backend = "gpu"  # or 'cpu'
from jax._src import config
config.update("jax_platforms", {"cpu": "cpu", "gpu": "cuda", "tpu": ""}[timesfm_backend])

In [87]:


import ta
from ta.trend import MACD, SMAIndicator
from ta.momentum import RSIIndicator, StochasticOscillator


# تنظیمات
ticker = 'AAPL'  # Example: Apple Inc.
start_date = "2022-10-12"
end_date = "2024-01-01"
analysis_start_date = "2023-01-03"  # تاریخ شروع مورد نظر

# دانلود داده‌های سهام از Yahoo Finance
data = yf.download(ticker, start=start_date, end=end_date)

# محاسبه نشانگرهای تکنیکال
data['macd'] = MACD(data['Close']).macd()
data['sma_50'] = SMAIndicator(data['Close'], window=50).sma_indicator()
data['stochastic'] = StochasticOscillator(data['Close'], data['High'], data['Low']).stoch()

# محاسبه Bollinger Bands به صورت دستی
data['bollinger_mid'] = data['Close'].rolling(window=20).mean()
data['bollinger_std'] = data['Close'].rolling(window=20).std()
data['bollinger_hband'] = data['bollinger_mid'] + (data['bollinger_std'] * 2)
data['bollinger_lband'] = data['bollinger_mid'] - (data['bollinger_std'] * 2)




# Calculate moving averages
data['moving_avg_10'] = data['Close'].rolling(window=10).mean()
data['moving_avg_50'] = data['Close'].rolling(window=50).mean()
data['rsi'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
data['ema'] = ta.trend.EMAIndicator(data['Close'], window=10).ema_indicator()

# Drop rows with NaN values (caused by rolling window calculations)
data = data.dropna()
data = data[data.index >= analysis_start_date]
# بروزرسانی DataFrame
df = data[['Close', 'Volume', 'moving_avg_10', 'moving_avg_50', 'rsi', 'ema', 'macd', 'bollinger_hband', 'bollinger_lband', 'stochastic']].reset_index()
df.columns = ['date', 'close', 'volume', 'moving_avg_10', 'moving_avg_50', 'rsi', 'ema', 'macd', 'bollinger_hband', 'bollinger_lband', 'stochastic']


df.set_index('date', inplace=True)
df








[*********************100%%**********************]  1 of 1 completed


,close,volume,moving_avg_10,moving_avg_50,rsi,ema,macd,bollinger_hband,bollinger_lband,stochastic
date,,,,,,,,,,
2023-01-03,125.070000,112117500,130.488998,142.709599,32.304320,130.829227,-4.588133,149.775583,122.660417,-37.242250
2023-01-04,126.360001,89113600,129.887999,142.291399,34.891363,130.016640,-4.678850,148.513103,121.895897,-24.604818
2023-01-05,125.019997,80962700,129.159998,141.802799,33.460886,129.108160,-4.803500,147.840951,120.779048,-34.478733
2023-01-06,129.619995,87754700,128.576998,141.348399,42.218538,129.201221,-4.479468,147.052434,120.435564,-37.499963
2023-01-09,130.149994,70790800,128.368998,140.964399,43.147003,129.373725,-4.132269,145.826902,120.411096,27.604201
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,193.600006,37122800,195.747002,185.398801,54.609714,194.957922,2.636317,199.490535,187.846466,-11.255516
2023-12-26,193.050003,28919300,195.734003,185.682801,53.026883,194.611027,2.306308,199.372064,188.290938,-14.285809
2023-12-27,193.149994,48087700,195.578001,185.971400,53.291938,194.345385,2.029448,199.282852,188.655150,-51.948305


In [88]:
import yfinance as yf

# دانلود اطلاعات شرکت
ticker = 'AAPL'  # مثال: Apple Inc.
stock = yf.Ticker(ticker)

# دریافت اطلاعات استاتیک
market_cap = stock.info.get('marketCap')
sector = stock.info.get('sector')
industry = stock.info.get('industry')
employees = stock.info.get('fullTimeEmployees')
country = stock.info.get('country')
exchange = stock.info.get('exchange')
currency = stock.info.get('currency')

# نمایش اطلاعات استاتیک
print(f"Market Cap: {market_cap}")
print(f"Sector: {sector}")
print(f"Industry: {industry}")
print(f"Employees: {employees}")
print(f"Country: {country}")
print(f"Exchange: {exchange}")
print(f"Currency: {currency}")


Market Cap: 3287734812672
Sector: Technology
Industry: Consumer Electronics
Employees: 161000
Country: United States
Exchange: NMS
Currency: USD


In [90]:
static_numerical_covariates = {
    'market_cap': [market_cap],
    'employees': [employees]
}

static_categorical_covariates = {
    'sector': [sector],
    'industry': [industry],
    'country': [country],
    'exchange': [exchange],
    'currency': [currency]
}




In [94]:


def get_batched_data_fn(batch_size: int = 32, context_len: int = 32, horizon_len: int = 1):

    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(df) - (context_len + horizon_len), horizon_len):
        num_examples += 1
        context_end = start + context_len
        examples["inputs"].append(df["close"][start:context_end].tolist())
        # اضافه کردن متغیرهای کمکی
        examples["volume"].append(df["volume"][start:context_end].tolist() + [df["volume"].iloc[context_end - 1]])
        examples["moving_avg_10"].append(df["moving_avg_10"][start:context_end].tolist() + [df["moving_avg_10"].iloc[context_end - 1]])
        examples["moving_avg_50"].append(df["moving_avg_50"][start:context_end].tolist() + [df["moving_avg_50"].iloc[context_end - 1]])
        examples["rsi"].append(df["rsi"][start:context_end].tolist() + [df["rsi"].iloc[context_end - 1]])
        examples["ema"].append(df["ema"][start:context_end].tolist() + [df["ema"].iloc[context_end - 1]])
        examples["macd"].append(df["macd"][start:context_end].tolist() + [df["macd"].iloc[context_end - 1]])
        examples["bollinger_hband"].append(df["bollinger_hband"][start:context_end].tolist() + [df["bollinger_hband"].iloc[context_end - 1]])
        examples["bollinger_lband"].append(df["bollinger_lband"][start:context_end].tolist() + [df["bollinger_lband"].iloc[context_end - 1]])
        examples["stochastic"].append(df["stochastic"][start:context_end].tolist() + [df["stochastic"].iloc[context_end - 1]])
        examples["outputs"].append(df["close"][context_end:context_end + horizon_len].tolist())
        examples["dates"].append(df.index[start:context_end + horizon_len].tolist())  # Add dates to examples


    def data_fn():
        for i in range(1 + (num_examples - 1) // batch_size):
            yield {k: v[(i * batch_size): ((i + 1) * batch_size)] for k, v in examples.items()}

    return data_fn

In [92]:


# Define metrics
def mse(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.square(y_pred - y_true), axis=1, keepdims=True)

def mae(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.abs(y_pred - y_true), axis=1, keepdims=True)

In [93]:
import timesfm

context_len = 64
batch_size = 1  # ثابت نگه‌داشتن batch size
horizon_len = 1  # ثابت نگه‌داشتن horizon length



In [10]:

    model = timesfm.TimesFm(
        context_len=context_len,
        horizon_len=horizon_len,
        input_patch_len=32,
        output_patch_len=128,
        num_layers=20,
        model_dims=1280,
        backend="gpu",
    )
    model.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 3.70 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.40 seconds.
Jitting decoding.
Jitted decoding in 24.22 seconds.


In [95]:
import os
import time
from collections import defaultdict
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import timesfm
import ta
from itertools import combinations
import gc
import json



features = ['volume', 'moving_avg_10', 'moving_avg_50', 'rsi', 'ema', 'macd', 'bollinger_hband', 'bollinger_lband', 'stochastic']
feature_combinations = []
for r in range(1, 4):
    feature_combinations.extend(combinations(features, r))
results_by_cov = []

for combo in feature_combinations:
    print(f"Processing combo: {combo}")
    input_data = get_batched_data_fn(batch_size=batch_size, context_len=context_len, horizon_len=1)
    metrics = defaultdict(list)


    # Process in smaller sub-batches
    for i, example in enumerate(input_data()):
        dynamic_numerical_covariates = {feature: example[feature] for feature in combo}
        raw_forecast, _ = model.forecast(
                            inputs=example["inputs"], freq=[0] * len(example["inputs"])
                        )
        cov_forecast, ols_forecast = model.forecast_with_covariates(
           inputs=example["inputs"],
           dynamic_numerical_covariates=dynamic_numerical_covariates,
           dynamic_categorical_covariates={},
           static_numerical_covariates=static_numerical_covariates,
           static_categorical_covariates=static_categorical_covariates,
           freq=[0] * len(example["inputs"]),
           normalize_xreg_target_per_input=True,
           xreg_mode="timesfm + xreg",)


        for j in range(len(example["inputs"])):
            result = {
                'input_index': j,
                'train_start_date': str(example["dates"][j][0]),
                'train_end_date': str(example["dates"][j][-2]),
                'predict_date': str(example["dates"][j][-1]),
                'raw_forecast': raw_forecast[j][0],
                'y_predict' : cov_forecast[j][0],
                'y_actual': example["outputs"][j][0],
                'batch_size': batch_size,
                'context_len': context_len,
                "timesfm + xreg":"timesfm + xreg",
                'combo' : combo,

            }
            results_by_cov.append(result)


Processing combo: ('volume',)
Processing combo: ('moving_avg_10',)
Processing combo: ('moving_avg_50',)
Processing combo: ('rsi',)
Processing combo: ('ema',)
Processing combo: ('macd',)
Processing combo: ('bollinger_hband',)
Processing combo: ('bollinger_lband',)
Processing combo: ('stochastic',)
Processing combo: ('volume', 'moving_avg_10')
Processing combo: ('volume', 'moving_avg_50')
Processing combo: ('volume', 'rsi')
Processing combo: ('volume', 'ema')
Processing combo: ('volume', 'macd')
Processing combo: ('volume', 'bollinger_hband')
Processing combo: ('volume', 'bollinger_lband')
Processing combo: ('volume', 'stochastic')
Processing combo: ('moving_avg_10', 'moving_avg_50')
Processing combo: ('moving_avg_10', 'rsi')
Processing combo: ('moving_avg_10', 'ema')
Processing combo: ('moving_avg_10', 'macd')
Processing combo: ('moving_avg_10', 'bollinger_hband')
Processing combo: ('moving_avg_10', 'bollinger_lband')
Processing combo: ('moving_avg_10', 'stochastic')
Processing combo: (

In [97]:
 results_df = pd.DataFrame(results_by_cov)
 results_df

,input_index,train_start_date,train_end_date,predict_date,raw_forecast,y_predict,y_actual,batch_size,context_len,timesfm + xreg,combo
0,0,2023-01-03 00:00:00,2023-04-04 00:00:00,2023-04-05 00:00:00,166.959579,174.092316,163.759995,1,64,timesfm + xreg,"(volume,)"
1,0,2023-01-04 00:00:00,2023-04-05 00:00:00,2023-04-06 00:00:00,165.057465,175.042053,164.660004,1,64,timesfm + xreg,"(volume,)"
2,0,2023-01-05 00:00:00,2023-04-06 00:00:00,2023-04-10 00:00:00,165.524918,181.442673,162.029999,1,64,timesfm + xreg,"(volume,)"
3,0,2023-01-06 00:00:00,2023-04-10 00:00:00,2023-04-11 00:00:00,162.622696,177.749969,160.800003,1,64,timesfm + xreg,"(volume,)"
4,0,2023-01-09 00:00:00,2023-04-11 00:00:00,2023-04-12 00:00:00,161.525681,175.524048,160.100006,1,64,timesfm + xreg,"(volume,)"
...,...,...,...,...,...,...,...,...,...,...,...
23860,0,2023-09-21 00:00:00,2023-12-20 00:00:00,2023-12-21 00:00:00,195.240768,218.109497,194.679993,1,64,timesfm + xreg,"(bollinger_hband, bollinger_lband, stochastic)"
23861,0,2023-09-22 00:00:00,2023-12-21 00:00:00,2023-12-22 00:00:00,194.696442,216.400131,193.600006,1,64,timesfm + xreg,"(bollinger_hband, bollinger_lband, stochastic)"
23862,0,2023-09-25 00:00:00,2023-12-22 00:00:00,2023-12-26 00:00:00,194.012909,213.656998,193.050003,1,64,timesfm + xreg,"(bollinger_hband, bollinger_lband, stochastic)"
23863,0,2023-09-26 00:00:00,2023-12-26 00:00:00,2023-12-27 00:00:00,193.501068,212.477951,193.149994,1,64,timesfm + xreg,"(bollinger_hband, bollinger_lband, stochastic)"


In [98]:
# Define functions for calculating accuracy and profit
def calculate_accuracy(y_actual, y_pred):
    correct_predictions = 0
    total_predictions = 0

    for i in range(1, len(y_actual)):
        actual_today = y_actual[i]
        predicted_today = y_pred[i]
        actual_yesterday = y_actual[i - 1]

        actual_movement = actual_today - actual_yesterday
        predicted_movement = predicted_today - actual_yesterday

        if (actual_movement > 0 and predicted_movement > 0) or (actual_movement < 0 and predicted_movement < 0):
            correct_predictions += 1
        total_predictions += 1

    accuracy_value = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy_value

def calculate_profit(y_actual, y_pred, initial_investment=100):
    capital = initial_investment
    shares = 0

    for i in range(1, len(y_actual)):
        if y_pred[i] > y_pred[i - 1] and capital > 0:
            shares = capital / y_actual[i]
            capital = 0
        elif y_pred[i] < y_pred[i - 1] and shares > 0:
            capital = shares * y_actual[i]
            shares = 0

    final_capital = capital + shares * y_actual[-1]
    profit = final_capital - initial_investment
    return profit

# Calculate metrics for each combo
final_results = []

for combo in feature_combinations:
    combo_results = results_df[results_df['combo'] == combo]

    y_actual = combo_results['y_actual'].values
    raw_forecast = combo_results['y_predict'].values

    # Calculate metrics
    mse_value = mean_squared_error(y_actual, raw_forecast)
    accuracy_value = calculate_accuracy(y_actual, raw_forecast)
    profit_value = calculate_profit(y_actual, raw_forecast)

    final_results.append({
        'combo': combo,
        'mse': mse_value,
        'accuracy': accuracy_value,
        'profit': profit_value
    })

# Convert final results to DataFrame
final_results_df = pd.DataFrame(final_results)

# Display the results
print(final_results_df)



                                              combo         mse  accuracy  \
0                                         (volume,)  109.683253  0.478261   
1                                  (moving_avg_10,)  293.098329  0.467391   
2                                  (moving_avg_50,)  288.539212  0.461957   
3                                            (rsi,)  134.720092  0.494565   
4                                            (ema,)  308.695638  0.472826   
..                                              ...         ...       ...   
124              (ema, bollinger_lband, stochastic)  293.463757  0.494565   
125        (macd, bollinger_hband, bollinger_lband)  272.241058  0.478261   
126             (macd, bollinger_hband, stochastic)  268.906302  0.489130   
127             (macd, bollinger_lband, stochastic)  269.404150  0.489130   
128  (bollinger_hband, bollinger_lband, stochastic)  289.821492  0.489130   

        profit  
0     5.043303  
1    17.037306  
2    14.389107  
3     6

In [99]:
best_mse_combo = final_results_df.loc[final_results_df['mse'].idxmin()]
best_accuracy_combo = final_results_df.loc[final_results_df['accuracy'].idxmax()]
best_profit_combo = final_results_df.loc[final_results_df['profit'].idxmax()]

print(f"Best MSE Combo: {best_mse_combo['combo']} with MSE: {best_mse_combo['mse']}")
print(f"Best Accuracy Combo: {best_accuracy_combo['combo']} with Accuracy: {best_accuracy_combo['accuracy']}")
print(f"Best Profit Combo: {best_profit_combo['combo']} with Profit: {best_profit_combo['profit']}")


Best MSE Combo: ('volume',) with MSE: 109.68325257213358
Best Accuracy Combo: ('rsi', 'macd', 'bollinger_lband') with Accuracy: 0.5108695652173914
Best Profit Combo: ('macd', 'bollinger_hband', 'bollinger_lband') with Profit: 22.86577213811121


In [101]:
import pickle

# Save results_by_cov to a .pkl file
with open('time_fm_xreg_fnacy_cov.pkl', 'wb') as f:
    pickle.dump(results_by_cov, f)

print("results_by_cov has been saved to 'results_by_cov.pkl'.")


results_by_cov has been saved to 'results_by_cov.pkl'.


In [102]:
# محاسبه‌ی متریک‌ها برای پیش‌بینی خام (Raw Forecast)
results_df=results_df[results_df['combo'] == combo]
y_actual = results_df['y_actual'].values
raw_forecast = results_df['raw_forecast'].values

# محاسبه MSE برای پیش‌بینی خام
raw_mse = mean_squared_error(y_actual, raw_forecast)

# محاسبه Accuracy برای پیش‌بینی خام
raw_accuracy = calculate_accuracy(y_actual, raw_forecast)

# محاسبه Profit برای پیش‌بینی خام
raw_profit = calculate_profit(y_actual, raw_forecast)

# نمایش نتایج برای پیش‌بینی خام
print(f"Raw Forecast MSE: {raw_mse}")
print(f"Raw Forecast Accuracy: {raw_accuracy}")
print(f"Raw Forecast Profit: {raw_profit}")


Raw Forecast MSE: 5.4792340349909425
Raw Forecast Accuracy: 0.45652173913043476
Raw Forecast Profit: 12.098695224770552
